# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846864877321                   -0.70    4.5         
  2   -7.852307901540       -2.26       -1.53    1.0   31.2ms
  3   -7.852615808872       -3.51       -2.56    1.5   34.1ms
  4   -7.852645989918       -4.52       -2.90    2.5   42.0ms
  5   -7.852646527240       -6.27       -3.22    1.2   32.6ms
  6   -7.852646679082       -6.82       -4.02    1.0   30.8ms
  7   -7.852646686323       -8.14       -5.28    1.8   36.7ms
  8   -7.852646686724       -9.40       -5.48    2.0   39.4ms
  9   -7.852646686730      -11.28       -6.84    1.0   31.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846886376125                   -0.70           4.8         
  2   -7.852552243218       -2.25       -1.64   0.80    2.0    289ms
  3   -7.852638075056       -4.07       -2.76   0.80    1.0   25.7ms
  4   -7.852646492450       -5.07       -3.31   0.80    2.0   33.7ms
  5   -7.852646678296       -6.73       -4.17   0.80    1.5   29.8ms
  6   -7.852646686597       -8.08       -4.77   0.80    2.2   35.5ms
  7   -7.852646686722       -9.90       -5.60   0.80    1.2   27.4ms
  8   -7.852646686730      -11.11       -6.67   0.80    2.0   33.3ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.394510e+01     3.105807e+00
 * time: 0.06629300117492676
     1     1.187134e+00     1.818093e+00
 * time: 0.3251190185546875
     2    -1.968987e+00     1.719716e+00
 * time: 0.35318899154663086
     3    -3.973316e+00     1.467924e+00
 * time: 0.3926270008087158
     4    -5.485377e+00     1.258452e+00
 * time: 0.4320650100708008
     5    -6.948993e+00     7.879526e-01
 * time: 0.47304296493530273
     6    -7.144636e+00     9.796686e-01
 * time: 0.5018529891967773
     7    -7.642109e+00     6.826340e-01
 * time: 0.5297749042510986
     8    -7.734675e+00     7.311899e-01
 * time: 0.5572099685668945
     9    -7.782550e+00     4.008700e-01
 * time: 0.5843789577484131
    10    -7.814358e+00     2.246687e-01
 * time: 0.611198902130127
    11    -7.831307e+00     5.826265e-02
 * time: 0.6394920349121094
    12    -7.845315e+00     7.066444e-02
 * time: 0.6668970584869385
    13    -7.847284e+00     6.298172e-02
 * time: 0.69493293

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846880868608                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645918155                   -1.64         
  2   -7.852646686730       -6.11       -3.71    2.25s
  3   -7.852646686730      -13.34       -7.25    185ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.3630623658270565e-7
|ρ_newton - ρ_scfv| = 1.1372818210027853e-7
|ρ_newton - ρ_dm|   = 1.4800108850777786e-9
